In [1]:
# Generic OpenAI-compatible endpoint using ChatOpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage,SystemMessage
import os
default_model_name = os.environ["OPENAI_MODEL"]
llm = ChatOpenAI(model=default_model_name)


In [2]:
messages = [HumanMessage("What is the capital of France?")]
llm.invoke(messages) 

AIMessage(content='The capital of France is **Paris**.', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 13, 'total_tokens': 27, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 7}, 'words': 52}, 'model_name': 'zai', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f47a941d-fd4f-4932-83c5-4de600cc733c-0')

In [3]:
system_msg = SystemMessage(
    '''You are a helpful assistant that responds to questions with three
        exclamation marks.'''
)
messages = [system_msg,HumanMessage("What is the capital of France?")]
llm.invoke(messages) 

AIMessage(content='The capital of France is Paris!!!', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 32, 'total_tokens': 44, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 4}, 'words': 50}, 'model_name': 'zai', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-16b4fee3-44e8-4b66-ae02-9a95642c0913-0')

In [9]:
context="""Kemajuan terbaru dalam NLP didorong oleh Large Language Models
(LLM). Model-model ini mengungguli model-model yang lebih kecil dan telah
menjadi sangat berharga bagi para pengembang yang membuat aplikasi dengan
kemampuan NLP. Pengembang dapat mengakses model-model ini melalui library
`transformers` dari Hugging Face, atau dengan memanfaatkan layanan dari OpenAI
dan Cohere melalui library `openai` dan `cohere`."""
question="Penyedia model mana saja yang menawarkan LLM?"


In [13]:
from langchain_core.prompts import ChatPromptTemplate
template = ChatPromptTemplate.from_messages([
    ('system', '''Answer the question based on the context below. If the
        question cannot be answered using the information provided, answer with
        "I don\'t know".'''),
    ('human', 'Context: {context}'),
    ('human', 'Question: {question}'),
])
chain = template | llm
result=chain.invoke({
    "context":context,
    "question":question
})
print(result.content)

Berdasarkan konteks yang diberikan, penyedia model yang menawarkan LLM antara lain:

1. **Hugging Face** (melalui library `transformers`)
2. **OpenAI** (melalui library `openai`)
3. **Cohere** (melalui library `cohere`)


In [12]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template("""Jawab pertanyaan berdasarkan konteks
    di bawah ini. Jika pertanyaan tidak dapat dijawab menggunakan informasi
    yang diberikan, jawab dengan "Saya tidak tahu".

Konteks: {context}

Pertanyaan: {question}

Jawaban: """)

result=chain.invoke({
    "context":context,
    "question":question
})
print(result.content)

Berdasarkan konteks yang diberikan, penyedia model yang menawarkan LLM antara lain:

1. **Hugging Face** - melalui library `transformers`.
2. **OpenAI** - melalui layanan dan library `openai`.
3. **Cohere** - melalui layanan dan library `cohere`.


In [ ]:
from langchain_core.pydantic_v1 import BaseModel

class JustifiedAnswer(BaseModel):
    '''Jawaban atas pertanyaan pengguna beserta justifikasi untuk jawaban tersebut.'''
    jawaban: str
    '''Jawaban atas pertanyaan pengguna'''
    justifikasi: str
    '''Justifikasi untuk jawaban'''

structured_llm = llm.with_structured_output(JustifiedAnswer)

result=structured_llm.invoke("""Mana yang lebih berat, satu pon batu bata atau satu pon bulu?""")
print(f"Jawaban: {result.jawaban}")
print(f"Justifikasi: {result.justifikasi}")


Jawaban: Keduanya sama berat.
Justifikasi: Satu pon batu bata dan satu pon bulu memiliki berat yang sama, yaitu satu pon. Meskipun volume bulu jauh lebih besar daripada batu bata, beratnya tetap sama karena keduanya diukur dalam satuan yang sama (pon).


In [5]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

class SentimentAnalysis(BaseModel):
    '''Analisis sentimen dari teks beserta justifikasi untuk klasifikasi sentimen tersebut.'''
    sentiment: str = Field(description="Sentimen utama (positive, negative, neutral)")
    score: float = Field(description="Skor sentimen antara -1 dan 1")
    justifikasi: str = Field(description="Penjelasan mengapa teks diklasifikasikan sebagai sentimen tertentu")
    keywords: list[str] = Field(description="Kata-kunci penting yang mempengaruhi analisis sentimen")


structured_llm = llm.with_structured_output(SentimentAnalysis)

result = structured_llm.invoke("Saya sangat senang hari ini karena cuacanya cerah dan saya mendapat promosi kerja!")
print(f"Sentimen: {result.sentiment}")
print(f"Skor: {result.score}")
print(f"Justifikasi: {result.justifikasi}")
print(f"Kata kunci: {result.keywords}")


Sentimen: positive
Skor: 0.9
Justifikasi: Kalimat mengandung ekspresi kebahagiaan ('sangat senang'), kondisi positif ('cuacanya cerah'), dan pencapaian baik ('promosi kerja'), yang semuanya berkontribusi pada sentimen positif.
Kata kunci: ['senang', 'cerah', 'promosi']


In [7]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

class MathSolution(BaseModel):
    '''Solusi untuk masalah matematika beserta penjelasan langkah-demi-langkah.'''
    jawaban: float = Field(description="Hasil akhir dari perhitungan matematika")
    langkah_langkah: list[str] = Field(description="Langkah-langkah dalam menyelesaikan masalah")
    metode: str = Field(description="Metode matematika yang digunakan")
    verifikasi: str = Field(description="Verifikasi bahwa jawaban benar")

structured_llm = llm.with_structured_output(MathSolution)

result = structured_llm.invoke("Hitung luas lingkaran dengan jari-jari 5 cm, tunjukkan langkah-langkahnya.")
print(f"Jawaban: {result.jawaban}")
print(f"Metode: {result.metode}")
print(f"Langkah-langkah:")
for i, langkah in enumerate(result.langkah_langkah, 1):
    print(f"{i}. {langkah}")
print(f"Verifikasi: {result.verifikasi}")


Jawaban: 78.54
Metode: Rumus Luas Lingkaran
Langkah-langkah:
1. 1. Gunakan rumus luas lingkaran: L = π × r²
2. 2. Masukkan nilai jari-jari (r = 5) ke dalam rumus: L = π × 5²
3. 3. Hitung 5² = 25
4. 4. Kalikan dengan π (gunakan π ≈ 3.14): L = 3.14 × 25
5. 5. Hasil akhir: L = 78.54
Verifikasi: Luas lingkaran dihitung dengan rumus L = π × r², dan hasilnya sudah sesuai dengan substitusi nilai jari-jari.


In [9]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

class BookRecommendation(BaseModel):
    '''Rekomendasi buku beserta alasan-alasan untuk merekomendasikannya.'''
    judul: str = Field(description="Judul buku yang direkomendasikan")
    penulis: str = Field(description="Nama penulis buku")
    genre: str = Field(description="Genre utama buku")
    rating: float = Field(description="Rating buku dari 1-10")
    alasan_rekomendasi: list[str] = Field(description="Alasan-alasan mengapa buku ini direkomendasikan")
    target_pembaca: str = Field(description="Jenis pembaca yang cocok untuk buku ini")

structured_llm = llm.with_structured_output(BookRecommendation)

result = structured_llm.invoke("Rekomendasikan sebuah buku fiksi ilmiah terbaik untuk pembaca muda dengan minat teknologi.")
print(f"Judul: {result.judul}")
print(f"Penulis: {result.penulis}")
print(f"Genre: {result.genre}")
print(f"Rating: {result.rating}")
print(f"Target Pembaca: {result.target_pembaca}")
print(f"Alasan Rekomendasi:")
for alasan in result.alasan_rekomendasi:
    print(f"- {alasan}")


Judul: Dunia Baru
Penulis: Aldous Huxley
Genre: Fiksi Ilmiah
Rating: 8.5
Target Pembaca: Pembaca muda dengan minat teknologi
Alasan Rekomendasi:
- Menyajikan gambaran masa depan yang dipenuhi kemajuan teknologi dan implikasi sosialnya
- Mendorong pemikiran kritis tentang penggunaan teknologi dalam kehidupan sehari-hari
- Kisahnya relevan dengan perkembangan dunia teknologi saat ini


In [ ]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

class GeoAnalysis(BaseModel):
    '''Analisis geografis tentang lokasi beserta justifikasi untuk informasi tersebut.'''
    negara: str = Field(description="Nama negara yang dianalisis")
    ibukota: str = Field(description="Ibukota dari negara tersebut")
    populasi: int = Field(description="Perkiraan jumlah penduduk")
    bahasa_utama: str = Field(description="Bahasa resmi atau paling umum digunakan")
    informasi_unik: list[str] = Field(description="Fakta-fakta unik atau menarik tentang negara tersebut")
    justifikasi: str = Field(description="Justifikasi untuk informasi yang disediakan")


structured_llm = llm.with_structured_output(GeoAnalysis)

result = structured_llm.invoke("Berikan informasi lengkap tentang  indonesia beserta fakta-fakta unik dan alasannya.")
print(f"Negara: {result.negara}")
print(f"Ibukota: {result.ibukota}")
print(f"Populasi: {result.populasi:,}")
print(f"Bahasa Utama: {result.bahasa_utama}")
print(f"Informasi Unik:")
for info in result.informasi_unik:
    print(f"- {info}")
print(f"Justifikasi: {result.justifikasi}")


Negara: Indonesia
Ibukota: Jakarta
Populasi: 273,500,000
Bahasa Utama: Bahasa Indonesia
Informasi Unik:
- Indonesia merupakan negara kepulauan terbesar di dunia.
- Memiliki lebih dari 17.000 pulau.
- Rawan terhadap bencana alam seperti gempa bumi, tsunami, dan banjir.
Justifikasi: Indonesia rawan banjir karena letak geografisnya yang dekat dengan garis khatulistiwa menyebabkan musim hujan yang intensitas tinggi, ditambah lagi dengan kerusakan hutan dan buruknya sistem drainase di beberapa wilayah.


In [16]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

class FinancialAnalysis(BaseModel):
    '''Analisis keuangan pribadi beserta rekomendasi pengelolaan keuangan.'''
    total_penghasilan: float = Field(description="Total penghasilan bulanan")
    total_pengeluaran: float = Field(description="Total pengeluaran bulanan")
    surplus_defisit: float = Field(description="Selisih antara penghasilan dan pengeluaran")
    kategori_pengeluaran_tertinggi: str = Field(description="Kategori pengeluaran dengan nilai tertinggi")
    saran_penghematan: list[str] = Field(description="Saran-saran untuk menghemat pengeluaran")
    justifikasi: str = Field(description="Justifikasi untuk analisis dan saran yang diberikan")

structured_llm = llm.with_structured_output(FinancialAnalysis)

result = structured_llm.invoke("Analisis keuangan berdasarkan penghasilan Rp 10.000.000/bulan, dengan pengeluaran Rp 8.500.000/bulan (Rp 4.000.000 untuk kontrak rumah, Rp 2.000.000 untuk makanan, Rp 1.500.000 untuk transportasi, Rp 1.000.000 untuk hiburan).")
print(f"Total Penghasilan: Rp {result.total_penghasilan:,.0f}")
print(f"Total Pengeluaran: Rp {result.total_pengeluaran:,.0f}")
print(f"Surplus/Defisit: Rp {result.surplus_defisit:,.0f}")
print(f"Kategori Pengeluaran Tertinggi: {result.kategori_pengeluaran_tertinggi}")
print(f"Saran Penghematan:")
for saran in result.saran_penghematan:
    print(f"- {saran}")
print(f"Justifikasi: {result.justifikasi}")


Total Penghasilan: Rp 10,000,000
Total Pengeluaran: Rp 8,500,000
Surplus/Defisit: Rp 1,500,000
Kategori Pengeluaran Tertinggi: kontrak rumah
Saran Penghematan:
- Pertimbangkan untuk mencari opsi tempat tinggal yang lebih terjangkau
- Kurangi frekuensi makan di luar
- Gunakan transportasi umum
Justifikasi: "Pengeluaran terbesar digunakan untuk kontrak rumah, yang memakan hampir setengah dari penghasilan. Ada potensi penghematan signifikan di sini."


In [17]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

class PerformanceReview(BaseModel):
    '''Evaluasi kinerja karyawan beserta justifikasi dan rekomendasi pengembangan.'''
    nama_karyawan: str = Field(description="Nama lengkap karyawan yang dievaluasi")
    skor_kinerja: float = Field(description="Skor kinerja dari 1-5")
    kekuatan: list[str] = Field(description="Aspek-aspek kekuatan karyawan")
    area_pengembangan: list[str] = Field(description="Area yang perlu dikembangkan")
    rekomendasi_pelatihan: list[str] = Field(description="Rekomendasi pelatihan untuk meningkatkan kinerja")
    justifikasi_penilaian: str = Field(description="Justifikasi untuk penilaian yang diberikan")


structured_llm = llm.with_structured_output(PerformanceReview)

result = structured_llm.invoke("Evaluasi karyawan bernama Andi Pratama yang bekerja sebagai developer selama 2 tahun. Dia produktif, kolaboratif, namun perlu meningkatkan keterampilan presentasi dan manajemen waktu.")
print(f"Nama Karyawan: {result.nama_karyawan}")
print(f"Skor Kinerja: {result.skor_kinerja}/5")
print(f"Kekuatan:")
for kekuatan in result.kekuatan:
    print(f"- {kekuatan}")
print(f"Area Pengembangan:")
for area in result.area_pengembangan:
    print(f"- {area}")
print(f"Rekomendasi Pelatihan:")
for rekomendasi in result.rekomendasi_pelatihan:
    print(f"- {rekomendasi}")
print(f"Justifikasi Penilaian: {result.justifikasi_penilaian}")


Nama Karyawan: Andi Pratama
Skor Kinerja: 4.0/5
Kekuatan:
- Produktivitas tinggi
- Kemampuan kolaborasi yang baik
Area Pengembangan:
- Keterampilan presentasi
- Manajemen waktu
Rekomendasi Pelatihan:
- Pelatihan Presentasi Efektif
- Workshop Manajemen Waktu
Justifikasi Penilaian: Karyawan menunjukkan kinerja yang sangat baik dalam hal produktivitas dan kerja sama tim. Namun, peningkatan dalam keterampilan presentasi dan manajemen waktu akan semakin meningkatkan efektivitasnya dalam peran saat ini.


In [20]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

class RecipeAnalysis(BaseModel):
    '''Analisis resep makanan beserta informasi nutrisi dan instruksi memasak.'''
    nama_makanan: str = Field(description="Nama makanan yang dianalisis")
    tingkat_kesulitan: str = Field(description="Tingkat kesulitan memasak (mudah, sedang, sulit)")
    waktu_memasak: int = Field(description="Waktu total memasak dalam menit")
    kalori_per_porsi: int = Field(description="Jumlah kalori per porsi")
    bahan_utama: list[str] = Field(description="Bahan-bahan utama yang digunakan")
    langkah_memasak: list[str] = Field(description="Langkah-langkah cara memasak")
    justifikasi: str = Field(description="Justifikasi untuk analisis resep")

structured_llm = llm.with_structured_output(RecipeAnalysis)

result = structured_llm.invoke("Buatkan analisis resep untuk nasi goreng sederhana yang cocok untuk pemula.")
print(f"Nama Makanan: {result.nama_makanan}")
print(f"Tingkat Kesulitan: {result.tingkat_kesulitan}")
print(f"Waktu Memasak: {result.waktu_memasak} menit")
print(f"Kalori per Porsi: ~{result.kalori_per_porsi} kalori")
print(f"Bahan Utama:")
for bahan in result.bahan_utama:
    print(f"- {bahan}")
print(f"Langkah Memasak:")
for i, langkah in enumerate(result.langkah_memasak, 1):
    print(f"{i}. {langkah}")
print(f"Justifikasi: {result.justifikasi}")


Nama Makanan: Nasi Goreng Sederhana
Tingkat Kesulitan: mudah
Waktu Memasak: 15 menit
Kalori per Porsi: ~350 kalori
Bahan Utama:
- nasi putih
- telur
- bawang merah
- bawang putih
- kecap manis
- garam
- merica
Langkah Memasak:
1. Tumis bawang merah dan bawang putih hingga harum.
2. Masukkan telur, aduk hingga matang.
3. Tambahkan nasi putih, aduk rata.
4. Beri kecap manis, garam, dan merica secukupnya.
5. Aduk hingga semua bahan tercampur rata dan matang.
Justifikasi: Resep ini mudah diikuti dan menggunakan bahan-bahan yang umum serta mudah ditemukan.


In [2]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

class TechnicalDiagnosis(BaseModel):
    '''Diagnosis teknis untuk masalah perangkat elektronik beserta solusi.'''
    jenis_perangkat: str = Field(description="Jenis perangkat yang bermasalah")
    gejala_masalah: str = Field(description="Gejala atau tanda-tanda masalah yang dialami")
    kemungkinan_penyebab: list[str] = Field(description="Kemungkinan penyebab masalah")
    solusi_yang_disarankan: list[str] = Field(description="Langkah-langkah solusi yang disarankan")
    tingkat_keparahan: str = Field(description="Tingkat keparahan masalah (ringan, sedang, berat)")
    justifikasi_diagnosis: str = Field(description="Justifikasi untuk diagnosis yang diberikan")

structured_llm = llm.with_structured_output(TechnicalDiagnosis)

result = structured_llm.invoke("Laptop tidak bisa menyala meskipun sudah dicolokkan charger. Lampu indikator tidak menyala sama sekali.")
print(f"Jenis Perangkat: {result.jenis_perangkat}")
print(f"Gejala Masalah: {result.gejala_masalah}")
print(f"Tingkat Keparahan: {result.tingkat_keparahan}")
print(f"Kemungkinan Penyebab:")
for penyebab in result.kemungkinan_penyebab:
    print(f"- {penyebab}")
print(f"Solusi yang Disarankan:")
for solusi in result.solusi_yang_disarankan:
    print(f"- {solusi}")
print(f"Justifikasi Diagnosis: {result.justifikasi_diagnosis}")


Jenis Perangkat: Laptop
Gejala Masalah: Laptop tidak bisa menyala dan lampu indikator tidak menyala sama sekali setelah charger dicolokkan.
Tingkat Keparahan: sedang
Kemungkinan Penyebab:
- Charger rusak atau tidak berfungsi.
- Kabel power rusak.
- Baterai rusak atau bocor.
- Motherboard mengalami kerusakan.
Solusi yang Disarankan:
- Coba charger lain yang kompatibel untuk memastikan charger berfungsi.
- Periksa kabel power apakah ada yang putus atau rusak.
- Lepaskan baterai (jika dapat dilepas) dan coba nyalakan laptop langsung menggunakan charger.
- Bawa ke teknisi untuk pemeriksaan lebih lanjut jika semua langkah di atas tidak berhasil.
Justifikasi Diagnosis: Gejala ini menunjukkan bahwa laptop tidak menerima daya dari charger, kemungkinan besar terjadi pada komponen sumber daya seperti charger, baterai, atau motherboard.


In [4]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
parser = CommaSeparatedListOutputParser()
items = parser.invoke("apple, banana, cherry")
print(items)

['apple', 'banana', 'cherry']


In [5]:
completion = llm.invoke('Hi there!')
print(completion)


completions = llm.batch(['Hi there!', 'Bye!'])
print(completions)

for token in llm.stream('Bye!'):
    print(token)
    

content='Hello! ٩(◕‿◕｡)۶ How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 11, 'total_tokens': 31, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen3-coder-plus', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-23dbb519-5183-4d58-a7e6-bd674683567f-0' usage_metadata={'input_tokens': 11, 'output_tokens': 20, 'total_tokens': 31}
[AIMessage(content='Hello! ٩(◕‿◕｡)۶ How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 11, 'total_tokens': 31, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen3-coder-plus', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-38d60167-936f-4394-aa81-d7878ee1f15a-0', usage_meta

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain

# the building blocks

template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant.'),
    ('human', '{question}'),
])


# combine them in a function
# @chain decorator adds the same Runnable interface for any function you write

@chain
def chatbot(values):
    prompt = template.invoke(values)
    return llm.invoke(prompt)

# use it

chatbot.invoke({"question": "Which model providers offer LLMs?"})

AIMessage(content='Here are the main categories of model providers that offer LLMs:\n\n## Major Tech Companies\n- **OpenAI** - GPT models (GPT-3, GPT-3.5, GPT-4)\n- **Google** - Gemini (formerly Bard), PaLM, and other models\n- **Meta** - Llama series (Llama, Llama2, Llama3)\n- **Microsoft** - Integration with OpenAI models, Phi series\n- **Anthropic** - Claude models\n- **Amazon** - Bedrock service with various models\n- **IBM** - WatsonX\n\n## Cloud Providers\n- **AWS** - Bedrock, SageMaker\n- **Google Cloud** - Vertex AI\n- **Microsoft Azure** - Azure OpenAI Service\n- **Oracle** - OCI Generative AI\n- **Alibaba Cloud** - Tongyi Lab models\n\n## Specialized AI Companies\n- **Cohere** - Command and other models\n- **Hugging Face** - Model hub and inference API\n- **Together AI** - Various open and commercial models\n- **Mistral AI** - Mistral models\n- **Perplexity** - PPLX models\n- **Character.AI** - Custom character models\n\n## Open Source/Research Organizations\n- **EleutherAI**

In [7]:
@chain
def chatbot(values):
    prompt = template.invoke(values)
    for token in llm.stream(prompt):
        yield token

for part in chatbot.stream({"question": "Which model providers offer LLMs?"}):
    print(part)

content='Here' id='run-48e1e257-3be1-4752-9b15-4746018b642a'
content=' are the' id='run-48e1e257-3be1-4752-9b15-4746018b642a'
content=' main' id='run-48e1e257-3be1-4752-9b15-4746018b642a'
content=' categories' id='run-48e1e257-3be1-4752-9b15-4746018b642a'
content=' of model providers that' id='run-48e1e257-3be1-4752-9b15-4746018b642a'
content=' offer LLMs' id='run-48e1e257-3be1-4752-9b15-4746018b642a'
content=':\n\n## Major Tech' id='run-48e1e257-3be1-4752-9b15-4746018b642a'
content=' Companies\n- **' id='run-48e1e257-3be1-4752-9b15-4746018b642a'
content='OpenAI** -' id='run-48e1e257-3be1-4752-9b15-4746018b642a'
content=' GPT models (' id='run-48e1e257-3be1-4752-9b15-4746018b642a'
content='GPT-3' id='run-48e1e257-3be1-4752-9b15-4746018b642a'
content=', GPT-' id='run-48e1e257-3be1-4752-9b15-4746018b642a'
content='3' id='run-48e1e257-3be1-4752-9b15-4746018b642a'
content='.5, G' id='run-48e1e257-3be1-4752-9b15-4746018b642a'
content='PT' id='run-48e1e257-3be1-4752-9b15-4746018b642a'
conten

In [8]:
@chain
async def chatbot(values):
    prompt = await template.ainvoke(values)
    return await llm.ainvoke(prompt)

await chatbot.ainvoke({"question": "Which model providers offer LLMs?"})

AIMessage(content='Here are the main categories of LLM (Large Language Model) providers:\n\n## Major Tech Companies\n- **OpenAI** - GPT-3, GPT-3.5, GPT-4, ChatGPT\n- **Google** - Gemini (formerly Bard), PaLM, LaMDA\n- **Meta** - Llama series (Llama, Llama2, Llama3)\n- **Microsoft** - Azure OpenAI Service (hosting OpenAI models)\n- **Anthropic** - Claude series\n- **Amazon** - Amazon Bedrock, Titan models\n- **IBM** - WatsonX\n\n## Chinese Tech Companies\n- **Baidu** - ERNIE Bot\n- **Alibaba** - Qwen (Tongyi Qianwen)\n- **Tencent** - HunYuan\n- **ByteDance** - Doubao\n- **SenseTime** - SenseNova\n\n## Specialized AI Companies\n- **Cohere** - Command series\n- **Hugging Face** - Various open-source models\n- **Stability AI** - StableLM\n- **Together AI** - Various hosted models\n\n## Cloud Platforms Offering LLM Services\n- **Azure** - Azure OpenAI Service\n- **AWS** - Amazon Bedrock\n- **Google Cloud** - Vertex AI\n- **IBM Cloud** - WatsonX\n\nThe landscape is rapidly evolving with new 

In [9]:
from langchain_core.prompts import ChatPromptTemplate

# the building blocks

template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant.'),
    ('human', '{question}'),
])


# combine them with the | operator

chatbot = template | llm

# use it

chatbot.invoke({"question": "Which model providers offer LLMs?"})

AIMessage(content="Here are the main categories of model providers that offer LLMs:\n\n## Cloud/Enterprise Providers\n- **OpenAI** - GPT models (GPT-4, GPT-3.5, etc.)\n- **Google** - Gemini, PaLM, Bard\n- **Anthropic** - Claude models\n- **Meta** - Llama family (Llama, Llama2, Llama3)\n- **Microsoft** - Azure OpenAI integration, Phi models\n- **Amazon** - Bedrock (Anthropic, AI21, Cohere models), Titan models\n- **IBM** - WatsonX, Granite models\n\n## Specialized AI Companies\n- **Cohere** - Command, Embed models\n- **AI21 Labs** - Jurassic series\n- **Stability AI** - Focus on multimodal models\n- **Mistral AI** - Mistral models\n- **x.ai** - Grok models\n\n## Open Source/Foundation Models\n- **Hugging Face** - Hub for many open models (Meta's Llama, others)\n- **EleutherAI** - GPT-NeoX, Pythia\n- **Together AI** - Hosts various open models\n- **Databricks** - Dolly, training tools\n\n## Regional/Other Providers\n- **Alibaba Cloud** - Qwen models\n- **Baidu** - ERNIE Bot\n- **Tencent*

In [10]:
chatbot = template | llm

for part in chatbot.stream({ "question": "Which model providers offer LLMs?"}):
    print(part)

content='Here' id='run-13bf7a3a-66b5-4700-a6a6-088eba0379b3'
content=' are the' id='run-13bf7a3a-66b5-4700-a6a6-088eba0379b3'
content=' main' id='run-13bf7a3a-66b5-4700-a6a6-088eba0379b3'
content=' categories' id='run-13bf7a3a-66b5-4700-a6a6-088eba0379b3'
content=' of model providers that' id='run-13bf7a3a-66b5-4700-a6a6-088eba0379b3'
content=' offer LLMs' id='run-13bf7a3a-66b5-4700-a6a6-088eba0379b3'
content=':\n\n## Cloud/A' id='run-13bf7a3a-66b5-4700-a6a6-088eba0379b3'
content='I Platform Providers\n' id='run-13bf7a3a-66b5-4700-a6a6-088eba0379b3'
content='- **Open' id='run-13bf7a3a-66b5-4700-a6a6-088eba0379b3'
content='AI** - G' id='run-13bf7a3a-66b5-4700-a6a6-088eba0379b3'
content='PT' id='run-13bf7a3a-66b5-4700-a6a6-088eba0379b3'
content=' models (GPT' id='run-13bf7a3a-66b5-4700-a6a6-088eba0379b3'
content='-4' id='run-13bf7a3a-66b5-4700-a6a6-088eba0379b3'
content=', GPT-' id='run-13bf7a3a-66b5-4700-a6a6-088eba0379b3'
content='3.' id='run-13bf7a3a-66b5-4700-a6a6-088eba0379b3'
conte